In [1]:
# 모듈 호출
import cv2
import numpy as np

In [2]:
# YOLO 호출
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [3]:
# 유형별 계층 개수 확인
for t in net.getLayerTypes():
    print('%s 유형 계층: %d개' % (t, net.getLayersCount(t)))

BatchNorm 유형 계층: 72개
Concat 유형 계층: 2개
Convolution 유형 계층: 75개
Eltwise 유형 계층: 23개
Identity 유형 계층: 2개
Permute 유형 계층: 3개
ReLU 유형 계층: 72개
Region 유형 계층: 3개
Resize 유형 계층: 2개
__NetInputLayer__ 유형 계층: 1개


In [6]:
# 계층 이름 출력
layer_names = net.getLayerNames()
print(layer_names)

['conv_0', 'bn_0', 'relu_1', 'conv_1', 'bn_1', 'relu_2', 'conv_2', 'bn_2', 'relu_3', 'conv_3', 'bn_3', 'relu_4', 'shortcut_4', 'conv_5', 'bn_5', 'relu_6', 'conv_6', 'bn_6', 'relu_7', 'conv_7', 'bn_7', 'relu_8', 'shortcut_8', 'conv_9', 'bn_9', 'relu_10', 'conv_10', 'bn_10', 'relu_11', 'shortcut_11', 'conv_12', 'bn_12', 'relu_13', 'conv_13', 'bn_13', 'relu_14', 'conv_14', 'bn_14', 'relu_15', 'shortcut_15', 'conv_16', 'bn_16', 'relu_17', 'conv_17', 'bn_17', 'relu_18', 'shortcut_18', 'conv_19', 'bn_19', 'relu_20', 'conv_20', 'bn_20', 'relu_21', 'shortcut_21', 'conv_22', 'bn_22', 'relu_23', 'conv_23', 'bn_23', 'relu_24', 'shortcut_24', 'conv_25', 'bn_25', 'relu_26', 'conv_26', 'bn_26', 'relu_27', 'shortcut_27', 'conv_28', 'bn_28', 'relu_29', 'conv_29', 'bn_29', 'relu_30', 'shortcut_30', 'conv_31', 'bn_31', 'relu_32', 'conv_32', 'bn_32', 'relu_33', 'shortcut_33', 'conv_34', 'bn_34', 'relu_35', 'conv_35', 'bn_35', 'relu_36', 'shortcut_36', 'conv_37', 'bn_37', 'relu_38', 'conv_38', 'bn_38', 'r

In [8]:
# 출력 계층 이름 출력
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(outputlayers)

['yolo_82', 'yolo_94', 'yolo_106']


In [12]:
# 80개 객체 유형 색상 정의
colors= np.random.uniform(0,255,size=(len(classes),3))

In [59]:
img = cv2.imread("D:/GEODATA/family.jpg") # 영상 읽기
img = cv2.resize(img,None,fx=1,fy=1) # 영상 크기, 비율(가로, 세로) 조절
height,width,channels = img.shape # 영상 높이, 너비, 채널

In [60]:
# 영상보기
cv2.imshow("Family", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [61]:
# 이미지에서 4차원 블롭 생성
blob = cv2.dnn.blobFromImage(img, # 영상
                             0.00392, # 축척계수
                             (416,416), # 공간크기
                             (0,0,0), # 평균차감
                             True,# RB 채널 교체
                             crop=False) # 자르기

In [62]:
net.setInput(blob) # 네트워크 입력값 설정
outs = net.forward(outputlayers) # 네트워크 순방향 실행

In [63]:
# 객체 탐지
class_ids=[]
confidences=[]
boxes=[]
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores) # 객체
        confidence = scores[class_id] # 신뢰도
        if confidence > 0.5:
            center_x= int(detection[0]*width)
            center_y= int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)
        
            cv2.circle(img,(center_x,center_y),10,(0,255,0),2) # 원 그리기
            
            x=int(center_x - w/2)
            y=int(center_y - h/2)
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2) # 사각형 그리기
            
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

cv2.imshow("Family",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [72]:
indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)

font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        cv2.putText(img,label,(x,y+30),font,1,(255,255,255),2)
            
cv2.imshow("Family",img)
cv2.waitKey(0)
cv2.destroyAllWindows()